# Space

In [1]:
import os
import sys 
import logging
import random
import pandas as pd 
from pprint import pprint 
from IPython.display import display, HTML

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
# pprint(SPACE)

# Available Packages
import argparse
import datasets
import pandas as pd
from datetime import datetime 


from recfldtkn.configfn import load_cohort_args
from recfldtkn.loadtools import load_module_variables, update_args_to_list
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer
from config_observer.CKPD import ckpd_to_CkpdObsConfig

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace


# [Part 1] Get a Caseset: Case Examples

## [Step 1]

In [2]:
from recfldtkn.loadtools import fetch_TriggerEvent_tools


####################
# TriggerCaseMethod = 'CGM5MinEntry'
TriggerCaseMethod = 'FoodEntryEOD'

####################

Trigger_tools = fetch_TriggerEvent_tools(TriggerCaseMethod, SPACE)
Trigger_tools 

{'TriggerRecName': 'FoodRec',
 'case_id_columns': ['PID', 'ObsDT'],
 'special_columns': ['PID', 'DT_s'],
 'convert_TriggerEvent_to_Caseset': <function FoodEntryEOD.convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config)>}

In [3]:
##################################
CaseSetName = TriggerCaseMethod
case_id_columns = Trigger_tools['case_id_columns']
special_columns = Trigger_tools['special_columns']
##################################

TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)

cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
cohort_args['case_id_columns'] = case_id_columns
cohort_args['ObsDTName'] = 'ObsDT'
cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

print(cohort_args)
print(TriggerCasePath)

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

In [4]:
CaseSetName = 'CGM5MinEntry'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case1 = pd.read_pickle(TriggerCasePath) # (1000)

CaseSetName = 'FoodEntryEOD'
TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
df_case2 = pd.read_pickle(TriggerCasePath) # (1000)

df_case = pd.merge(df_case1, df_case2, on=case_id_columns, how='inner')
# df_case = df_case1.sample(1000, random_state=0).reset_index(drop = True)

df_case

,PID,ObsDT
0,1000037,2021-01-07
1,1000037,2021-01-08
2,1000048,2021-01-11
3,1000048,2021-01-13
4,1000048,2021-01-14
...,...,...
2701,3000039,2023-08-06
2702,3000039,2023-08-07
2703,3000039,2023-08-08
2704,3000039,2023-08-11


In [5]:
# ##################################
# CaseSetName = TriggerCaseMethod
# case_id_columns = Trigger_tools['case_id_columns']
# special_columns = Trigger_tools['special_columns']
# ##################################

# TriggerCasePath = os.path.join(SPACE['DATA_CaseSet'], f'{CaseSetName}.p')
# cohort_args = load_cohort_args(recfldtkn_config_path, SPACE)

# cohort_args['ckpd_to_CkpdObsConfig'] = ckpd_to_CkpdObsConfig
# cohort_args['case_id_columns'] = case_id_columns
# cohort_args['ObsDTName'] = 'ObsDT'
# cohort_args['PID_ObsDT_columns'] = [cohort_args['RootID'], cohort_args['ObsDTName']]

# print(cohort_args)
# print(TriggerCasePath)

# df_case = pd.read_pickle(TriggerCasePath) # (1000)
# df_case = df_case.sample(1000, random_state=0).reset_index(drop=True)
ds_case = datasets.Dataset.from_pandas(df_case)
ds_case

Dataset({
    features: ['PID', 'ObsDT'],
    num_rows: 2706
})


## [Step 2] RO: Record Observation & CasePhi


In [6]:
###########################
ROName_List = ['FoodRec-Bf24H-NutriN2C']

name_CasePhi = 'nsSparseTknIn5MinTS' # non-seq tkn in 5min time step.
###########################

In [7]:
from recfldtkn.obsname import convert_RONameList_to_COName

CaseObsName = convert_RONameList_to_COName(ROName_List, name_CasePhi)
print(CaseObsName)

ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS


In [8]:
# part of fetch_caseobs_Phi_tools
CO_Folder = os.path.join(SPACE['DATA_CaseObs'], CaseObsName)
if not os.path.exists(CO_Folder): os.makedirs(CO_Folder)
print(CO_Folder)

../_Data/2-Data_CaseObs\ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS


# [Part 2] Tools for ds_case_obs

## [Step 1] Prepare examples

In [9]:
case_examples = ds_case[:5]
print(case_examples)

{'PID': [1000037, 1000037, 1000048, 1000048, 1000048], 'ObsDT': [Timestamp('2021-01-07 00:00:00'), Timestamp('2021-01-08 00:00:00'), Timestamp('2021-01-11 00:00:00'), Timestamp('2021-01-13 00:00:00'), Timestamp('2021-01-14 00:00:00')]}


In [10]:
length = len(case_examples[list(case_examples.keys())[0]])
case_examples_list = [{k: v[i] for k, v in case_examples.items()} for i in range(length)]
idx_to_examples = {i: case_examples_list[i] for i in range(length)}
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-07 00:00:00')},
 1: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-08 00:00:00')},
 2: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-11 00:00:00')},
 3: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-13 00:00:00')},
 4: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-14 00:00:00')}}


In [11]:
case_example = idx_to_examples[0]
case_example

{'PID': 1000037, 'ObsDT': Timestamp('2021-01-07 00:00:00')}

## [Step 2] get_caseobs_id

In [12]:
import inspect
#################################################
def get_CO_id(case_example, CaseObsName, cohort_args):
    RootID = cohort_args['RootID']
    ObsDTName = cohort_args['ObsDTName']
    PIDValue = case_example[RootID]
    ObsDTValue = case_example[ObsDTName].isoformat()
    return f'{PIDValue}&{ObsDTValue}'

get_CO_id.fn_string = inspect.getsource(get_CO_id)
#################################################

In [13]:
caseobs_id = get_CO_id(case_example, CaseObsName, cohort_args)
caseobs_id # CO_id

'1000037&2021-01-07T00:00:00'



## [Step 3] RO List and CasePhi

In [14]:
name_CasePhi 

'nsSparseTknIn5MinTS'

In [15]:
######### RecObs_Name
print(ROName_List) # RO_List
# Let's focus on one RO
ROName = ROName_List[0]
print(ROName)

['FoodRec-Bf24H-NutriN2C']
FoodRec-Bf24H-NutriN2C


## [Step4] RO Information

In [16]:
# decompose RO to RecName, CkpdName, FldName (small phi)
# decompose RO to RecName, CkpdName, FldName (small phi)
from recfldtkn.obsname import parse_RecObsName

d = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)
RecName = d['RecName']
CkpdName = d['CkpdName']
FldName = d['FldName']
print(RecName, CkpdName, FldName)

FoodRec Bf24H NutriN2C


In [17]:
RecName

'FoodRec'

In [18]:
# load all information about the R^record_name. 
from recfldtkn.configfn import load_record_args

print(RecName)
rec_args = load_record_args(RecName, cohort_args)
[i for i in rec_args]

FoodRec


['CohortInfo',
 'RawInfo',
 'RecName',
 'RecID',
 'RecIDChain',
 'DTCols',
 'RecDT',
 'ParentRecName',
 'attr_cols',
 'FldTknInfo',
 'RFT_GROUP_SIZE',
 'idx_group_size',
 'usebucket',
 'GROUP_SIZE',
 'pypath',
 'recfldtkn_config_path',
 'yaml_file_path']

## [Step 5] Load ds_rec

In [19]:
from recfldtkn.loadtools import load_ds_rec_and_info

ds_rec, ds_rec_info = load_ds_rec_and_info(RecName, cohort_args)
# R^Rx
# ROdf # Rx-Bf2M (i, t, Bf2M)

In [20]:
######### column_names (or we can all it attribute columns)
column_names = ds_rec.column_names # attr_columns + phi_columns
column_names
# ROdf_ij --(Phi: RecNum)-> COdf_ij 

['PID',
 'PatientID',
 'FoodRecID',
 'DT_s',
 'DT_r',
 'DT_tz',
 'EntryType',
 'FoodName',
 'ServingSize',
 'ServingType',
 'ServingsConsumed',
 'Carbs',
 'Fiber',
 'Fat',
 'Calories',
 'Protein',
 'Sodium',
 'SaturatedFat',
 'PolyUnSaturatedFat',
 'MonoUnSaturatedFat',
 'TransFat',
 'Cholesterol',
 'Potassium',
 'Sugar',
 'AddedSugars',
 'ActivityTypeID',
 'ExternalSourceID',
 'ObservationStatus',
 'FoodRec-SysCateTkn_wgt',
 'FoodRec-SysCateTkn_tknidx',
 'FoodRec-NutriN2CTkn_wgt',
 'FoodRec-NutriN2CTkn_tknidx',
 'FoodRec-TypeCateTkn_wgt',
 'FoodRec-TypeCateTkn_tknidx',
 'FoodRec-NutriNumeTkn_wgt',
 'FoodRec-NutriNumeTkn_tknidx']

## [Step 6] get_selected_columns (attr + phi) from ds_rec

In [21]:
column_names

['PID',
 'PatientID',
 'FoodRecID',
 'DT_s',
 'DT_r',
 'DT_tz',
 'EntryType',
 'FoodName',
 'ServingSize',
 'ServingType',
 'ServingsConsumed',
 'Carbs',
 'Fiber',
 'Fat',
 'Calories',
 'Protein',
 'Sodium',
 'SaturatedFat',
 'PolyUnSaturatedFat',
 'MonoUnSaturatedFat',
 'TransFat',
 'Cholesterol',
 'Potassium',
 'Sugar',
 'AddedSugars',
 'ActivityTypeID',
 'ExternalSourceID',
 'ObservationStatus',
 'FoodRec-SysCateTkn_wgt',
 'FoodRec-SysCateTkn_tknidx',
 'FoodRec-NutriN2CTkn_wgt',
 'FoodRec-NutriN2CTkn_tknidx',
 'FoodRec-TypeCateTkn_wgt',
 'FoodRec-TypeCateTkn_tknidx',
 'FoodRec-NutriNumeTkn_wgt',
 'FoodRec-NutriNumeTkn_tknidx']

In [22]:
from recfldtkn.obsname import parse_RecObsName
# RecName = RecObs_Name.split('-')[0]
base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
base_columns

['PID', 'FoodRecID', 'DT_s']

In [23]:
RecName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['RecName']
FldName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['FldName']
RecFldTknName = RecName + '-' + FldName + 'Tkn'
# rec_args['FldTknInfo'][RecFldTknName]
RecFldTknName

'FoodRec-NutriN2CTkn'

In [24]:
###############################
def get_selected_columns(ROName, column_names, cohort_args, rec_args, CaseTkn):
    from recfldtkn.obsname import parse_RecObsName
    ckpd_to_CkpdObsConfig = cohort_args['ckpd_to_CkpdObsConfig']
    RecName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['RecName']
    FldName = parse_RecObsName(ROName, ckpd_to_CkpdObsConfig)['FldName']
    base_columns = [cohort_args['RootID'], rec_args['RecID'], rec_args['RecDT']]
    
    FldName_cols = [i for i in column_names if FldName in i]
    if len(FldName_cols) == 0: 
        print(f'No columns found for {FldName}')
        # parse_RecObsName(ROName)['FldName']
        RecFldTknName = RecName + '-' + FldName + 'Tkn'
        # rec_args['FldTknInfo'][RecFldTknName]
        
        val_cols = rec_args['FldTknInfo'][RecFldTknName]['value_cols']
        final_cols = base_columns + val_cols
        raise ValueError(f'No columns found for {FldName}. Using value columns {val_cols}')
    else:
        final_cols = base_columns + FldName_cols
    return final_cols

get_selected_columns.fn_string = inspect.getsource(get_selected_columns)
###############################

In [25]:
# big question here
# if small phi is not applied in the record level
# how do we deploy the tools of small phi within this process?
get_selected_columns(ROName, column_names, cohort_args, rec_args, name_CasePhi)

['PID',
 'FoodRecID',
 'DT_s',
 'FoodRec-NutriN2CTkn_wgt',
 'FoodRec-NutriN2CTkn_tknidx']

In [26]:
print(ROName_List)
print(name_CasePhi)
print(get_selected_columns)

['FoodRec-Bf24H-NutriN2C']
nsSparseTknIn5MinTS
<function get_selected_columns at 0x00000194B692A3E0>


In [27]:
# RecObsName_to_RecObsInfo # RO_to_ROinfo

from recfldtkn.observer import get_RecObsName_to_RecObsInfo

record_to_ds_rec = {}        # set this to empty dictionary, then we will load data from disk
record_to_ds_rec_info = {}   # set this to empty dictionary, then we will load data from disk
ROName_to_ROInfo = get_RecObsName_to_RecObsInfo(ROName_List, 
                                                name_CasePhi, 
                                                get_selected_columns,
                                                cohort_args, 
                                                cohort_args['ckpd_to_CkpdObsConfig'], 
                                                record_to_ds_rec, 
                                                record_to_ds_rec_info)
    
ROName = ROName_List[0] # Rx-Bf2M  
ROInfo = ROName_to_ROInfo[ROName] # RxInfo + R^Rx (with necessary columns) + Ckpd + FldTkn (phi)

[INFO:2024-04-21 09:07:40,127:(configfn.py@116 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline\config_recfldtkn/Record\FoodRec.yaml


In [28]:
[i for i in ROInfo]
# pprint(ROInfo)

['rec_args',
 'RecName',
 'CkpdName',
 'CkpdInfo',
 'FldName',
 'FldTknName',
 'FldIdx2Tkn',
 'fldtkn_args',
 'fld_tokenizer_fn',
 'run_fldtkn_on_the_fly',
 'ds_rec',
 'df_rec_info']

In [29]:
ROInfo['ds_rec'] # get_selected_columns

Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 142022
})

## [Step 7] get_CO_vocab

In [30]:
def get_CO_vocab(RecObsName_to_RecObsInfo):
    # phi_list
    RecFldName_list = list(set([RecObsInfo['RecName'] + '-' +RecObsInfo['FldName'] 
                                for RecObsName, RecObsInfo in RecObsName_to_RecObsInfo.items()
                                if RecObsInfo['FldName'] is not None ]
                                ))
    
    # print('RecFldName_list----->', RecFldName_list)
    # for current version: for any Phi, we only process one or zero phi. 
    assert len(RecFldName_list) <= 1

    ############################ tkn 
    RecObsName = [i for i in RecObsName_to_RecObsInfo][0]
    RecObsInfo = RecObsName_to_RecObsInfo[RecObsName]
    idx2tkn = RecObsInfo['FldIdx2Tkn'] + ['NoObs']   
    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab = {}
    CaseTknVocab['tid'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    CaseTknVocab['wgt'] = {} # we don't use wgt in this version


    CkpdInfo = RecObsInfo['CkpdInfo']
    TimeStepType = '5Min'
    StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
    EndIdx   = CkpdInfo[f'EndIdx{TimeStepType}']
    idx2tkn = [f'{TimeStepType}_{i}' for i in range(StartIdx, EndIdx + 1)]
    tid2tkn = {tid: tkn for tid, tkn in enumerate(idx2tkn)}
    tkn2tid = {tkn: tid for tid, tkn in tid2tkn.items()}
    CaseTknVocab['timestep'] = {'tid2tkn': tid2tkn, 'tkn2tid': tkn2tid}
    ############################
    
    return CaseTknVocab

get_CO_vocab.fn_string = inspect.getsource(get_CO_vocab)

In [31]:
CO_vocab = get_CO_vocab(ROName_to_ROInfo)
pprint(CO_vocab, sort_dicts=False)

{'tid': {'tid2tkn': {0: 'unk',
                     1: 'Calories:0~100',
                     2: 'Calories:0~100Level',
                     3: 'Carbs:0~10',
                     4: 'Carbs:0~10Level',
                     5: 'Cholesterol:0~20',
                     6: 'Cholesterol:0~20Level',
                     7: 'Fat:0~5',
                     8: 'Fat:0~5Level',
                     9: 'Fiber:0~1',
                     10: 'Fiber:0~1Level',
                     11: 'MonoUnSaturatedFat:0~100',
                     12: 'MonoUnSaturatedFat:0~100Level',
                     13: 'PolyUnSaturatedFat:0~1',
                     14: 'PolyUnSaturatedFat:0~1Level',
                     15: 'Potassium:0~100',
                     16: 'Potassium:0~100Level',
                     17: 'Protein:0~10',
                     18: 'Protein:0~10Level',
                     19: 'SaturatedFat:0~2',
                     20: 'SaturatedFat:0~2Level',
                     21: 'ServingSize:0~1',
              


## [Step 8] Fetch ds_rec

In [32]:
case_example = idx_to_examples[4]
case_example

{'PID': 1000048, 'ObsDT': Timestamp('2021-01-14 00:00:00')}

In [33]:
ROName

'FoodRec-Bf24H-NutriN2C'

In [34]:
pprint(idx_to_examples, sort_dicts=False)

{0: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-07 00:00:00')},
 1: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-08 00:00:00')},
 2: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-11 00:00:00')},
 3: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-13 00:00:00')},
 4: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-14 00:00:00')}}


In [35]:
for ROName, ROInfo in ROName_to_ROInfo.items():
    print(ROName, ':', [i for i in ROInfo])

FoodRec-Bf24H-NutriN2C : ['rec_args', 'RecName', 'CkpdName', 'CkpdInfo', 'FldName', 'FldTknName', 'FldIdx2Tkn', 'fldtkn_args', 'fld_tokenizer_fn', 'run_fldtkn_on_the_fly', 'ds_rec', 'df_rec_info']


In [36]:
# RO_to_ROdf: RecObsName_to_RecObsDS
from recfldtkn.observer import CaseObserverTransformer

get_Record_P = CaseObserverTransformer.get_Record_P
RecName_to_REC_P = get_Record_P(idx_to_examples, ROName_to_ROInfo)
for RecName, REC_P in RecName_to_REC_P.items():
    print(f'============ {RecName} ============')
    pprint(REC_P)

============ FoodRec ============
{1000037: Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 4
}),
 1000048: Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 152
})}


## [Step 9] Fetch ROdf: $R_i^{recname}$

In [37]:
get_idx_to_RecObsName_to_RecObsDS = CaseObserverTransformer.get_idx_to_RecObsName_to_RecObsDS
idx_to_ROName_to_ROds = get_idx_to_RecObsName_to_RecObsDS(idx_to_examples, ROName_to_ROInfo, RecName_to_REC_P)

for idx, case in idx_to_examples.items():
    print('\n====================')
    print('idx:', idx)
    print('case:', case)
    ROName_to_ROds = idx_to_ROName_to_ROds[idx]
    print('ROName_to_ROds:', ROName_to_ROds) # R_i^{RecNum, Ckpd}
    for ROName, ROds in ROName_to_ROds.items():
        print('ROName:', ROName)
        if ROds is None: 
            print('ROds is None')
            continue
        ROdf = ROds.to_pandas()
        print(ROdf[rec_args['RecDT']])
        # print(ROdf[rec_args['RecID']])  
    print('====================')


idx: 0
case: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-07 00:00:00')}
ROName_to_ROds: {'FoodRec-Bf24H-NutriN2C': Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 2
})}
ROName: FoodRec-Bf24H-NutriN2C
0   2021-01-06 16:50:00
1   2021-01-06 17:30:00
Name: DT_s, dtype: datetime64[ns]

idx: 1
case: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-08 00:00:00')}
ROName_to_ROds: {'FoodRec-Bf24H-NutriN2C': Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 2
})}
ROName: FoodRec-Bf24H-NutriN2C
0   2021-01-07 16:28:00
1   2021-01-07 16:28:00
Name: DT_s, dtype: datetime64[ns]

idx: 2
case: {'PID': 1000048, 'ObsDT': Timestamp('2021-01-11 00:00:00')}
ROName_to_ROds: {'FoodRec-Bf24H-NutriN2C': Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 1
})}
ROName: FoodRec-Bf24H-NutriN2C

In [38]:
[i for i in ROName_to_ROInfo]

['FoodRec-Bf24H-NutriN2C']

In [39]:
idx = 1
case_example = idx_to_examples[idx]
print('case_example:', case_example)
ROName_to_ROds = idx_to_ROName_to_ROds[idx] # RO_to_ROdf # RO_ij
print('RO_to_ROdf (RO_ij):', ROName_to_ROds)
print([i for i in ROName_to_ROds])

case_example: {'PID': 1000037, 'ObsDT': Timestamp('2021-01-08 00:00:00')}
RO_to_ROdf (RO_ij): {'FoodRec-Bf24H-NutriN2C': Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 2
})}
['FoodRec-Bf24H-NutriN2C']


# [Step 10]: Develop $\Phi$. 

In [40]:
ObsDTValue = case_example['ObsDT'] 
ObsDTValue

Timestamp('2021-01-08 00:00:00')

In [41]:
ROName_to_ROds

{'FoodRec-Bf24H-NutriN2C': Dataset({
     features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
     num_rows: 2
 })}

In [42]:
assert len(ROName_to_ROds) == 1 

In [43]:
RecObsName = [i for i in ROName_to_ROds][0] # RO
RecObsName

'FoodRec-Bf24H-NutriN2C'

In [44]:
RecObsDS   = ROName_to_ROds[RecObsName]
RecObsDS

Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 2
})

In [45]:
RecObsInfo = ROName_to_ROInfo[RecObsName] 
RecObsInfo

{'rec_args': {'CohortInfo': {'RawData2022_CGM': {'ELogFoodItem': {'raw_data_path': '../_Data/0-Data_Raw/RawData2022_CGM/05_02_2022_ELogFoodItem.csv',
     'RawRootID': 'PatientID',
     'RawName': 'ELogFoodItem'}},
   'RawData2023_CVSTDCAug': {'ELogFoodItem': {'raw_data_path': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/08_23_2023_ELogFoodItem.csv',
     'RawRootID': 'PatientID',
     'RawName': 'ELogFoodItem'}},
   'RawData2023_CVSDeRxAug': {'ELogFoodItem': {'raw_data_path': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/08_23_2023_ELogFoodItem.csv',
     'RawRootID': 'PatientID',
     'RawName': 'ELogFoodItem'}}},
  'RawInfo': {'ELogFoodItem': {'RawRootID': 'PatientID',
    'RawName': 'ELogFoodItem',
    'raw_columns': ['ELogFoodItemID',
     'PatientID',
     'CarbEntryID',
     'FoodName',
     'FoodID',
     'FoodImageID',
     'ServingSize',
     'ServingType',
     'ServingsConsumed',
     'Carbs',
     'Fiber',
     'Fat',
     'Calories',
     'Protein',
     'Sodium',
     'Satur

In [46]:
RecDT = RecObsInfo['rec_args']['RecDT'] 
RecDT

'DT_s'

In [47]:
CkpdInfo = RecObsInfo['CkpdInfo']
CkpdInfo

{'DistStartToPredDT': -24,
 'DistEndToPredDT': 0.01,
 'TimeUnit': 'h',
 'StartIdx5Min': -288,
 'EndIdx5Min': 0}

In [48]:
TimeStepType = '5Min'

StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
StartIdx

-288

In [49]:
TimeStepType = '5Min'
StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
EndIdx   = CkpdInfo[f'EndIdx{TimeStepType}']
timestep_index = range(StartIdx, EndIdx + 1) 

In [50]:
RO_ds = RecObsDS
RO_ds

Dataset({
    features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
    num_rows: 2
})

In [51]:
if RO_ds is not None:
    df = RO_ds.to_pandas()
    df['timestep'] = ((df[RecDT] - ObsDTValue).dt.total_seconds() / (60 * 5)).astype(int)
    # desired_range_df = pd.DataFrame({'timestep': timestep_index})
    # df = pd.merge(df, desired_range_df, on='timestep', how='right')
else:
    # df = pd.DataFrame({'timestep': timestep_index, 
    #                    'tknidx': [0] * len(timestep_index),
    #                    'wgt': [1] * len(timestep_index)
    #                    },
    #                    )
    df = None
df

,PID,FoodRecID,DT_s,FoodRec-NutriN2CTkn_wgt,FoodRec-NutriN2CTkn_tknidx,timestep
0,1000037,1000037-2,2021-01-07 16:28:00,"[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11...",-90
1,1000037,1000037-3,2021-01-07 16:28:00,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[35, 37, 208, 9, 7, 1, 17, 25, 19, 13, 11, 29,...",-90


In [52]:
ObsDTValue

Timestamp('2021-01-08 00:00:00')

In [53]:
case_example

{'PID': 1000037, 'ObsDT': Timestamp('2021-01-08 00:00:00')}

In [54]:
df

,PID,FoodRecID,DT_s,FoodRec-NutriN2CTkn_wgt,FoodRec-NutriN2CTkn_tknidx,timestep
0,1000037,1000037-2,2021-01-07 16:28:00,"[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11...",-90
1,1000037,1000037-3,2021-01-07 16:28:00,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[35, 37, 208, 9, 7, 1, 17, 25, 19, 13, 11, 29,...",-90


In [55]:
columns = ['tknidx', 'wgt', 'timestep']
# columns = [i for i in columns if i in df.columns]
df = df.rename(columns = {i: i.split('Tkn_')[-1] for i in df.columns if 'Tkn_' in i})
print(df.columns)
df = df[columns]
df

Index(['PID', 'FoodRecID', 'DT_s', 'wgt', 'tknidx', 'timestep'], dtype='object')


,tknidx,wgt,timestep
0,"[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11...","[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-90
1,"[35, 37, 208, 9, 7, 1, 17, 25, 19, 13, 11, 29,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-90


In [56]:
# df = df.fillna(0)
df

,tknidx,wgt,timestep
0,"[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11...","[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-90
1,"[35, 37, 208, 9, 7, 1, 17, 25, 19, 13, 11, 29,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",-90


In [57]:
# a = df['tknidx'].iloc[0]
# type(a)

In [58]:
import itertools 
import numpy as np 

def convert_timestep(x):
    name = x.name
    # print(x, x.name)
    # print(x)
    # print(type(x))
    x = [i for i in x if type(i) == np.ndarray or type(i) == list]
    # print(col)
    if len(x) == 0:
        if name == 'wgt':
            return [1]
        else:
            return [0] 
    else:
        return list(itertools.chain(*x))

grouped_df = df.groupby('timestep').agg({col: convert_timestep
                                         for col in df.columns if col != 'timestep'}).reset_index()
grouped_df

,timestep,tknidx,wgt
0,-90,"[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11...","[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ..."


In [59]:
output = grouped_df.to_dict(orient='list')
pprint(output, compact=True)# ['tknidx']

{'timestep': [-90],
 'tknidx': [[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27, 35,
             37, 208, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27]],
 'wgt': [[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0]]}


In [60]:
TimeStep = 5
TimeUnit = 'minutes'

CO = {
    'tid': output['tknidx'],
    'wgt': output['wgt'], 
    'timestep': output['timestep'],
    'timeinfo': ['ObsDTValue', 
                    # 'StartDateTime', 
                    # 'EndDateTime', 
                    'TimeStepType', 
                    'TimeStepSize',
                    'TimeUnit',
                    'StartIdx-To-EndIdx', 
                    # 'EndIdx'
                    ],
    'timevalues': [ObsDTValue.isoformat(), 
                    # StartDateTime.isoformat(),
                    # EndDateTime.isoformat(), 
                    str(TimeStepType), 
                    str(TimeStep),
                    str(TimeUnit),
                    str(StartIdx) + ':To:' + str(EndIdx), 
                    ]
}

pprint(CO, compact = True, sort_dicts=False)

{'tid': [[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27, 35, 37,
          208, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27]],
 'wgt': [[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0]],
 'timestep': [-90],
 'timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit',
              'StartIdx-To-EndIdx'],
 'timevalues': ['2021-01-08T00:00:00', '5Min', '5', 'minutes', '-288:To:0']}


In [61]:
import inspect

##################################
def fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab, cohort_args):
    # input: RecObsName_to_RecObsDS, RecObsName_to_RecObsInfo
    # output: CaseObservation
    
    import itertools 
    def convert_timestep(x):
        name = x.name
        # print(x, x.name)
        # print(x)
        # print(type(x))
        x = [i for i in x if type(i) == np.ndarray or type(i) == list]
        # print(col)
        if len(x) == 0:
            if name == 'wgt':
                return [1]
            else:
                return [0] 
        else:
            return list(itertools.chain(*x))


    ObsDTValue = case_example['ObsDT']   # T_ij value
    assert len(ROName_to_ROds) == 1 # assert that RecObsDS has at least one record   
    RecObsName = [i for i in ROName_to_ROds][0] # RO
    RecObsDS   = ROName_to_ROds[RecObsName]  # RO_ds(RecDT is not caseDT)
    RecObsInfo = ROName_to_ROInfo[RecObsName] # RecObsInfo
    RecDT = RecObsInfo['rec_args']['RecDT'] 

    CkpdInfo = RecObsInfo['CkpdInfo']  # Ckpd
    
    TimeStepType = '5Min'
    TimeStep = 5
    TimeUnit = 'minutes' 
    StartIdx = CkpdInfo[f'StartIdx{TimeStepType}']
    EndIdx   = CkpdInfo[f'EndIdx{TimeStepType}']
    timestep_index = range(StartIdx, EndIdx + 1) 
        
    # 1. get a subset of a Record Type: e.g., CGM5Min_Bf24H, with TknIdx.
    RO_ds = RecObsDS  
    

    if RO_ds is not None:
        df = RO_ds.to_pandas()
        df['timestep'] = ((df[RecDT] - ObsDTValue).dt.total_seconds() / (60 * 5)).astype(int)
        # desired_range_df = pd.DataFrame({'timestep': timestep_index})
        # df = pd.merge(df, desired_range_df, on='timestep', how='right')
        # df = df.infer_objects().fillna(0)
    else:
        # df = pd.DataFrame({'timestep': timestep_index})
        # df = pd.DataFrame({'timestep': timestep_index, 
        #                'tknidx': [0] * len(timestep_index),
        #                'wgt': [0] * len(timestep_index)},
        #                )
        CO = {
            # 'tid': [], # [[0]] * len(timestep_index), 
            # 'wgt': [], # [[1]] * len(timestep_index),
            'tid': [[0]], # None, # [[0]] * len(timestep_index), 
            'wgt': [[0.]], # None, # [[1]] * len(timestep_index),
            'timestep': [0],
            'timeinfo': ['ObsDTValue', 
                        # 'StartDateTime', 
                        # 'EndDateTime', 
                        'TimeStepType', 
                        'TimeStepSize',
                        'TimeUnit',
                        'StartIdx-To-EndIdx', 
                        # 'EndIdx'
                        ],
            'timevalues': [ObsDTValue.isoformat(), 
                            # StartDateTime.isoformat(),
                            # EndDateTime.isoformat(), 
                            str(TimeStepType), 
                            str(TimeStep),
                            str(TimeUnit),
                            str(StartIdx) + ':To:' + str(EndIdx), 
                            ]
        }
        return CO
    
    # if RO_ds is None: 
    #     new_index = range(StartIdx5Min, EndIdx5Min + 1)  # Include 24
    #     print('RO_ds is None')
    #     CO = {'tid': [0] * len(new_index)}
    #     return CO

    # df = RO_ds.to_pandas()
    # some requirements for df
    # each row is a 5Min, and each tknidx is just a list with one token.

    columns = ['tknidx', 'wgt', 'timestep']
    df = df.rename(columns = {i: i.split('Tkn_')[-1] for i in df.columns if 'Tkn_' in i})
    # df = df[columns].fillna(0.)

    grouped_df = df.groupby('timestep').agg({'wgt': convert_timestep, 'tknidx': convert_timestep}).reset_index()
    output = grouped_df.to_dict(orient='list')
    

    CO = {
        'tid': output['tknidx'],
        'wgt': output['wgt'], 
        'timestep': output['timestep'],
        'timeinfo': ['ObsDTValue', 
                        # 'StartDateTime', 
                        # 'EndDateTime', 
                        'TimeStepType', 
                        'TimeStepSize',
                        'TimeUnit',
                        'StartIdx-To-EndIdx', 
                        # 'EndIdx'
                        ],
        'timevalues': [ObsDTValue.isoformat(), 
                        # StartDateTime.isoformat(),
                        # EndDateTime.isoformat(), 
                        str(TimeStepType), 
                        str(TimeStep),
                        str(TimeUnit),
                        str(StartIdx) + ':To:' + str(EndIdx), 
                        ]
    }
    return CO
##################################

fn_CasePhi.fn_string = inspect.getsource(fn_CasePhi)

In [62]:
ROName_to_ROds

{'FoodRec-Bf24H-NutriN2C': Dataset({
     features: ['PID', 'FoodRecID', 'DT_s', 'FoodRec-NutriN2CTkn_wgt', 'FoodRec-NutriN2CTkn_tknidx'],
     num_rows: 2
 })}

In [63]:
CO = fn_CasePhi(case_example, ROName_to_ROds, ROName_to_ROInfo, CO_vocab, cohort_args)

pprint(CO, compact=True, sort_dicts = False)

{'tid': [[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27, 35, 37,
          208, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27]],
 'wgt': [[1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0]],
 'timestep': [-90],
 'timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit',
              'StartIdx-To-EndIdx'],
 'timevalues': ['2021-01-08T00:00:00', '5Min', '5', 'minutes', '-288:To:0']}


In [64]:
CO_vocab['tid']['tid2tkn']

{0: 'unk',
 1: 'Calories:0~100',
 2: 'Calories:0~100Level',
 3: 'Carbs:0~10',
 4: 'Carbs:0~10Level',
 5: 'Cholesterol:0~20',
 6: 'Cholesterol:0~20Level',
 7: 'Fat:0~5',
 8: 'Fat:0~5Level',
 9: 'Fiber:0~1',
 10: 'Fiber:0~1Level',
 11: 'MonoUnSaturatedFat:0~100',
 12: 'MonoUnSaturatedFat:0~100Level',
 13: 'PolyUnSaturatedFat:0~1',
 14: 'PolyUnSaturatedFat:0~1Level',
 15: 'Potassium:0~100',
 16: 'Potassium:0~100Level',
 17: 'Protein:0~10',
 18: 'Protein:0~10Level',
 19: 'SaturatedFat:0~2',
 20: 'SaturatedFat:0~2Level',
 21: 'ServingSize:0~1',
 22: 'ServingSize:0~1Level',
 23: 'ServingsConsumed:0~1',
 24: 'ServingsConsumed:0~1Level',
 25: 'Sodium:0~100',
 26: 'Sodium:0~100Level',
 27: 'Sugar:0~5',
 28: 'Sugar:0~5Level',
 29: 'TransFat:0~0',
 30: 'TransFat:0~0Level',
 31: 'Fiber:1~2',
 32: 'Fiber:1~2Level',
 33: 'PolyUnSaturatedFat:1~2',
 34: 'PolyUnSaturatedFat:1~2Level',
 35: 'ServingSize:1~2',
 36: 'ServingSize:1~2Level',
 37: 'ServingsConsumed:1~2',
 38: 'ServingsConsumed:1~2Level',
 39

## Save Files

In [65]:
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

prefix = ['import pandas as pd', 'import numpy as np']
fn_variables = [get_CO_id, get_selected_columns, get_CO_vocab, fn_CasePhi]
pycode = convert_variables_to_pystirng(fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(SPACE['CODE_FN'], 'fn_casephi', f'phi_{name_CasePhi}.py')
with open(pypath, 'w') as file: file.write(pycode)

# Create a HTML link and display it
full_path = os.path.join(WORKSPACE_PATH, pypath)
display(HTML(f'{pypath} <a href="{full_path}" target="_blank">Open File</a>'))

# Application

In [66]:
CaseObsName

'ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS'

In [67]:
from recfldtkn.observer import get_CaseObsInfo_for_a_CaseObsName

CaseObsInfo = get_CaseObsInfo_for_a_CaseObsName(CaseObsName,
                                                SPACE, 
                                                cohort_args, 
                                                record_to_ds_rec = {}, 
                                                record_to_ds_rec_info = {})
[i for i in CaseObsInfo]

[INFO:2024-04-21 09:07:41,055:(configfn.py@116 recfldtkn.configfn)]: file_path in load_fldtkn_args: ../pipeline\config_recfldtkn/Record\FoodRec.yaml


['RecObsName_List',
 'name_CasePhi',
 'get_selected_columns',
 'CaseObsName',
 'ROName_to_ROInfo',
 'fn_CasePhi',
 'get_CO_id',
 'CO_Folder',
 'CO_vocab']

In [68]:
import random 
from pprint import pprint
from datetime import datetime 
from recfldtkn.observer import get_RecObsName_to_RecObsInfo, CaseObserverTransformer


ro_to_ROName = {} # leave it empty for now 
ROName_to_ROInfo = CaseObsInfo['ROName_to_ROInfo']
name_CasePhi = CaseObsInfo['name_CasePhi']
fn_CasePhi = CaseObsInfo['fn_CasePhi']
CO_vocab = CaseObsInfo['CO_vocab']
get_CO_id = CaseObsInfo['get_CO_id']
CO_Folder = CaseObsInfo['CO_Folder']
COids = None 
use_CO_from_disk = False
batch_size = CaseObsInfo.get('batch_size', 1000)

fn_caseobs_Phi = CaseObserverTransformer(ro_to_ROName,
                                         ROName_to_ROInfo, 
                                         name_CasePhi, 
                                         fn_CasePhi, 
                                         CO_vocab, 
                                         get_CO_id,
                                         cohort_args,
                                         CO_Folder, 
                                         df_case,
                                         use_CO_from_disk)

# Check Cache Functions

In [69]:
fn_caseobs_Phi.new_COs

{}

In [70]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/2706 [00:00<?, ? examples/s]

Elipse Time:  0:00:14.684868
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt', 'timestep', 'timeinfo', 'timevalues'],
    num_rows: 2706
})


In [71]:
print(len(fn_caseobs_Phi.new_COs))

2706


In [72]:
start = datetime.now()
ds_casetkn = ds_case.map(fn_caseobs_Phi, 
                         batched = True, 
                         batch_size= batch_size, 
                         load_from_cache_file = False, 
                         new_fingerprint = CaseObsName)
end = datetime.now()
print('Elipse Time: ', end - start)
print(ds_casetkn)

Map:   0%|          | 0/2706 [00:00<?, ? examples/s]

Elipse Time:  0:00:03.108097
Dataset({
    features: ['PID', 'ObsDT', 'tid', 'wgt', 'timestep', 'timeinfo', 'timevalues'],
    num_rows: 2706
})


In [73]:
print(CaseObsName)
random_int = random.randint(0, len(ds_casetkn))
print(random_int)
pprint(ds_casetkn[random_int], sort_dicts=False, compact=True)
pprint(CO_vocab, sort_dicts=False, compact=True)

ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS
1483
{'PID': 1002306,
 'ObsDT': Timestamp('2021-08-08 00:00:00'),
 'tid': [[35, 37, 119, 120, 9, 7, 1, 17, 25, 19, 13, 11, 29, 5, 15, 27, 35, 49,
          3, 4, 9, 71, 72, 1, 2, 17, 18, 25, 26, 19, 20, 33, 34, 11, 12, 29,
          314, 315, 15, 16, 27, 28],
         [35, 37, 3, 4, 9, 7, 8, 1, 2, 17, 18, 25, 19, 13, 11, 29, 5, 15, 27]],
 'wgt': [[1.0, 1.0, 1.0, 0.2, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0, 1.0, 1.0, 0.04, 1.0, 1.0, 0.41, 1.0, 0.92, 1.0, 0.63,
          1.0, 0.94, 1.0, 0.99, 1.0, 0.22, 1.0, 0.03, 1.0, 1.0, 0.5, 1.0, 0.68,
          1.0, 0.08],
         [1.0, 1.0, 1.0, 0.9, 1.0, 1.0, 0.5, 1.0, 0.7, 1.0, 0.3, 1.0, 1.0, 1.0,
          1.0, 1.0, 1.0, 1.0, 1.0]],
 'timestep': [-192, -144],
 'timeinfo': ['ObsDTValue', 'TimeStepType', 'TimeStepSize', 'TimeUnit',
              'StartIdx-To-EndIdx'],
 'timevalues': ['2021-08-08T00:00:00', '5Min', '5', 'minutes', '-288:To:0']}
{'tid': {'tid2tkn': 

In [74]:
ds_casetkn[4]

{'PID': 1000048,
 'ObsDT': Timestamp('2021-01-14 00:00:00'),
 'tid': [[35,
   37,
   3,
   4,
   9,
   71,
   72,
   278,
   17,
   18,
   338,
   339,
   45,
   13,
   11,
   29,
   224,
   286,
   287,
   27,
   28,
   35,
   37,
   3,
   4,
   9,
   7,
   1,
   17,
   25,
   19,
   13,
   11,
   29,
   5,
   15,
   27,
   35,
   91,
   3,
   4,
   31,
   32,
   7,
   8,
   1,
   2,
   17,
   18,
   25,
   19,
   20,
   13,
   14,
   11,
   12,
   29,
   5,
   15,
   16,
   27,
   28,
   35,
   37,
   119,
   120,
   41,
   7,
   1,
   2,
   17,
   18,
   395,
   396,
   19,
   13,
   11,
   29,
   5,
   369,
   370,
   132],
  [35,
   49,
   3,
   4,
   31,
   7,
   1,
   2,
   17,
   18,
   25,
   26,
   19,
   13,
   11,
   29,
   5,
   286,
   287,
   27,
   28,
   35,
   23,
   24,
   119,
   120,
   9,
   7,
   1,
   2,
   17,
   18,
   25,
   19,
   13,
   11,
   29,
   5,
   15,
   27,
   35,
   49,
   3,
   4,
   9,
   7,
   8,
   1,
   2,
   17,
   18,
   25,
   26,
   19,


## Save

In [75]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [76]:
CO_Folder_data = fn_caseobs_Phi.CO_Folder_data 
CO_Folder_data

'../_Data/2-Data_CaseObs\\ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS\\data'

In [77]:
fn_caseobs_Phi.save_new_COs_to_disk(CO_Folder_data)

In [78]:
fn_caseobs_Phi.df_CO_info

,caseobs_idx_in_data
caseobs_id,


In [79]:
fn_caseobs_Phi.ds_CO_data

# Save Vocab

In [80]:
CO_vocab = fn_caseobs_Phi.CO_vocab
CO_vocab

{'tid': {'tid2tkn': {0: 'unk',
   1: 'Calories:0~100',
   2: 'Calories:0~100Level',
   3: 'Carbs:0~10',
   4: 'Carbs:0~10Level',
   5: 'Cholesterol:0~20',
   6: 'Cholesterol:0~20Level',
   7: 'Fat:0~5',
   8: 'Fat:0~5Level',
   9: 'Fiber:0~1',
   10: 'Fiber:0~1Level',
   11: 'MonoUnSaturatedFat:0~100',
   12: 'MonoUnSaturatedFat:0~100Level',
   13: 'PolyUnSaturatedFat:0~1',
   14: 'PolyUnSaturatedFat:0~1Level',
   15: 'Potassium:0~100',
   16: 'Potassium:0~100Level',
   17: 'Protein:0~10',
   18: 'Protein:0~10Level',
   19: 'SaturatedFat:0~2',
   20: 'SaturatedFat:0~2Level',
   21: 'ServingSize:0~1',
   22: 'ServingSize:0~1Level',
   23: 'ServingsConsumed:0~1',
   24: 'ServingsConsumed:0~1Level',
   25: 'Sodium:0~100',
   26: 'Sodium:0~100Level',
   27: 'Sugar:0~5',
   28: 'Sugar:0~5Level',
   29: 'TransFat:0~0',
   30: 'TransFat:0~0Level',
   31: 'Fiber:1~2',
   32: 'Fiber:1~2Level',
   33: 'PolyUnSaturatedFat:1~2',
   34: 'PolyUnSaturatedFat:1~2Level',
   35: 'ServingSize:1~2',
   36

In [81]:
CO_Folder_vocab = fn_caseobs_Phi.CO_Folder_vocab
CO_Folder_vocab

'../_Data/2-Data_CaseObs\\ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS\\vocab.p'

In [82]:
df_Vocab = pd.DataFrame({CaseObsName: CO_vocab})
df_Vocab.to_pickle(CO_Folder_vocab)
df_Vocab

,ro.FoodRec-Bf24H-NutriN2C_ct.nsSparseTknIn5MinTS
tid,"{'tid2tkn': {0: 'unk', 1: 'Calories:0~100', 2:..."
wgt,{}
timestep,"{'tid2tkn': {0: '5Min_-288', 1: '5Min_-287', 2..."
